In [1]:
# 导包
import cv2 as cv
import numpy as np

In [2]:
# 初始化设置图片文件位置
plate_file_path = 'images/restoration3.jpg'

In [3]:
# 0. 加载图片
plate_image = cv.imread(plate_file_path)

In [4]:
# cv.imshow('plate_image', plate_image)
# cv.waitKey()
# cv.destroyAllWindows()

In [5]:
# 1. 图片预处理
blured_image = cv.GaussianBlur(plate_image, (5, 5), 0)

gray_image = cv.cvtColor(blured_image, cv.COLOR_BGR2GRAY)

grad_x = cv.Sobel(gray_image, cv.CV_16S, 1, 0, ksize=3)

abs_grad_x = cv.convertScaleAbs(grad_x)

gray_image = cv.addWeighted(abs_grad_x, 1, 0, 0, 0)

is_success, threshold_image = cv.threshold(gray_image, 0, 255, cv.THRESH_OTSU)

kernel = cv.getStructuringElement(cv.MORPH_RECT, (10, 3))
morphology_imge = cv.morphologyEx(threshold_image, cv.MORPH_CLOSE, kernel)

contours, _ = cv.findContours(morphology_imge, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

In [6]:
# 获取倾斜的轮廓
contour = contours[0]
# 获取矩形特征描述的等值线区域，返回：中心点坐标、长和宽、旋转角度
rect = cv.minAreaRect(contour)
# 获取整数形式的 长、宽
rect_width, rect_height = np.int0(rect[1])
angle = np.abs(rect[2])
if rect_width > rect_height:
    temp = rect_width
    rect_width = rect_height
    rect_height = temp
    angle = 90 + angle    # 需要理解&修改
    
# boundingRect 用于获取与 等值线框（轮廓框）contour 的四个角点正交的矩形
# 返回 左上的坐标（x, y），宽（w）、高（h）
x, y, w, h = cv.boundingRect(contour)
bounding_image = plate_image[y:y+h, x:x+w]

enlarged_width = w * 3 // 2
enlarged_height = h * 3 // 2
enlarged_image = np.zeros((enlarged_height, enlarged_width, plate_image.shape[2]), dtype=plate_image.dtype)

x_in_enlarged = (enlarged_width - w) // 2
y_in_enlarged = (enlarged_height - h) // 2

roi_image = enlarged_image[y_in_enlarged:y_in_enlarged+h, x_in_enlarged:x_in_enlarged+w, :]
cv.addWeighted(roi_image, 0, bounding_image, 1, 0, roi_image)

# 开始旋转
new_center = (enlarged_width // 2, enlarged_height // 2)
# 旋转
transform_matrix = cv.getRotationMatrix2D(new_center, angle+270, 1.0)
# transform_image = cv.warpAffine(enlarged_image, transform_matrix, (enlarged_width, enlarged_height))
transform_image = cv.warpAffine(enlarged_image, transform_matrix, (enlarged_width, enlarged_height))

# 获取输出图
output_image = cv.getRectSubPix(transform_image, (rect_height, rect_width), new_center)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [7]:
cv.imshow('output_image', output_image)
cv.waitKey()
cv.destroyAllWindows()

32